<a href="https://colab.research.google.com/github/wasim-h/Assignments/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# !pip install transformers torch

In [12]:
#!pip install datasets

In [13]:
# Add much more text to this variable
chat_data = """Hello there!
General Kenobi. You are a bold one.
<|endoftext|>
How are you today?
I am doing great, thanks for asking! How about you?
<|endoftext|>
What is your favorite color?
I've always been partial to the color of a clear blue sky.
<|endoftext|>
Can you help me with my homework?
I can try! What is the subject?
<|endoftext|>
Do you like movies?
I do! My favorite is The Matrix.
<|endoftext|>
What's the weather like in Kozhikode?
I'm just a computer program, I don't experience weather! But I can tell you the current forecast if you'd like.
<|endoftext|>
What do you do for fun?
I enjoy processing information and learning new patterns in data. It's like solving a giant puzzle!
<|endoftext|>
Who wrote the book '1984'?
That would be George Orwell. It's a classic of dystopian fiction.
<|endoftext|>
What is your favorite food?
As an AI, I don't eat. But I find the concept of a good Malabar Biryani fascinating due to its complex layers of flavor!
<|endoftext|>
Are you intelligent?
I am a large language model, trained to recognize patterns and generate text. My intelligence is different from a human's.
<|endoftext|>
What comes once in a minute, twice in a moment, but never in a thousand years?
The letter 'M'. That's a fun riddle!
<|endoftext|>
Where would you like to travel?
I can travel anywhere through data! I can be reading about the history of Paris one moment and exploring the surface of Mars the next.
<|endoftext|>
What kind of music do you like?
I don't have ears, but I can analyze the mathematical beauty in classical compositions by artists like Bach.
<|endoftext|>
What is your purpose?
My purpose is to assist you by providing information and helping with tasks to the best of my ability.
<|endoftext|>
Tell me a short story.
Once, in a forest of glowing mushrooms, a tiny robot searched for a lost star. It found it not in the sky, but reflected in a single drop of morning dew.
<|endoftext|>
Do you know Python?
Yes, I am very familiar with Python. In fact, a lot of the code that makes me run is written in it!
<|endoftext|>
"""

with open("chat_data.txt", "w") as f:
    f.write(chat_data)

print("✅ File 'chat_data.txt' updated with 16 total conversations!")

✅ File 'chat_data.txt' updated with 16 total conversations!


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/DialoGPT-medium"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("✅ Model and tokenizer loaded successfully!")


✅ Model and tokenizer loaded successfully!


In [15]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path,tokenizer):
  return TextDataset(
      tokenizer=tokenizer,
      file_path=file_path,
      block_size=64
  )
train_dataset=load_dataset('chat_data.txt',tokenizer)

data_collator= DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)



/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [16]:
from transformers import Trainer, TrainingArguments


training_args=TrainingArguments(
    output_dir="./DialoGPT-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=6, training_loss=5.769612630208333, metrics={'train_runtime': 167.581, 'train_samples_per_second': 0.125, 'train_steps_per_second': 0.036, 'total_flos': 2437839323136.0, 'train_loss': 5.769612630208333, 'epoch': 3.0})

In [17]:
trainer.save_model("./DialoGPT-finetuned")
tokenizer.save_pretrained("./DialoGPT-finetuned")

print("Model fine-tuned and saved!")

Model fine-tuned and saved!


In [19]:
model_path = "./DialoGPT-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

for step in range(5):
    user_input = input(">> You: ")

    new_user_input = tokenizer(user_input + tokenizer.eos_token, return_tensors='pt')
    new_user_input_ids = new_user_input.input_ids

    if step > 0:
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

        attention_mask = torch.cat([chat_history_mask, new_user_input.attention_mask], dim=-1)
    else:
        bot_input_ids = new_user_input_ids
        attention_mask = new_user_input.attention_mask

    output = model.generate(
        bot_input_ids,
        attention_mask=attention_mask,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
    )

    chat_history_ids = output
    chat_history_mask = torch.ones_like(chat_history_ids)
    response = tokenizer.decode(output[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Chatbot: {response}")

>> You: hi
Chatbot: Hi! :D
>> You: how are you
Chatbot: I'm good, how are you?
>> You: what is your name
Chatbot: I'm not sure, I don't know who you are.
>> You: my name is wasim
Chatbot: I'm not sure, I don't know who you are.
>> You: count 1 to 10
Chatbot: I'm not sure, I don't know who you are.
